# Supervised Machine Learning 

This training module was developed by Oyemwenosa N. Avenbuan, Alexis Payton, MS, and Dr. Julia E. Rager

Spring 2023

## Introduction to Training Module
Machine learning is a field that has been around for decades, but has exploded in popularity and utility in recent years due to proliferation of big data. Through the building of models, machine learning has the ability to sift through and learn from large volumes of data and use that knowledge to solve problems. The challenges of big and high dimensional data as they pertain to environmental health and how machine learning can mitigate some of those challenges are discussed further in [Payton et. al](https://www.frontiersin.org/articles/10.3389/ftox.2023.1171175/full).

In this training module, we will explore:
+ Types of machine learning
+ Building/ training a supervised machine learning algorithm
+ Assessment of a supervised machine learning model's performance

Supervised machine learning will be explored using a geology dataset. This example dataset was generated by measuring inorganic Arsenic (iAs) concentrations and other variables in 713 private wells across North Carolina. Let's go ahead and view these data. 

### Installing required R packages
If you already have these packages installed, you can skip this step, or you can run the below code which checks installation status for you

In [1]:
if (!requireNamespace("readxl"))
  install.packages("readxl");
if (!requireNamespace("lubridate"))
  install.packages("lubridate");
if (!requireNamespace("tidyverse"))
  install.packages("tidyverse");
if (!requireNamespace("gtsummary"))
  install.packages("gtsummary");
if (!requireNamespace("caret"))
  install.packages("caret");
if (!requireNamespace("e1071"))
  install.packages("e1071");
if (!requireNamespace("Hmsic"))
  install.packages("Hmsic");
if (!requireNamespace("randomForest"))
  install.packages("randomForest");
if (!requireNamespace("pROC"))
  install.packages("pROC");
if (!requireNamespace("themis"))
  install.packages("themis");
if (!requireNamespace("rlang"))
  install.packages("rlang");

Loading required namespace: readxl

Loading required namespace: lubridate

Loading required namespace: tidyverse

Loading required namespace: gtsummary

Loading required namespace: caret

Loading required namespace: e1071

Loading required namespace: Hmsic

Warning message:
“package ‘Hmsic’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Loading required namespace: randomForest

Loading required namespace: themis



### Loading required R packages

In [2]:
library(readxl);
library(lubridate);
library(tidyverse);
library(gtsummary);
library(caret);
library(e1071);
library(Hmisc);
library(randomForest);
library(pROC);
library(themis);
library(rlang);


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.2     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.2     ✔ tibble  3.2.1
✔ purrr   1.0.1     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

### Set your working directory

In [ ]:
setwd("/filepath to where your input files are")

### Importing example dataset
**Will need to change input to module number and add module number to the file itself**

In [14]:
# Load the data
well_data <- data.frame(read_excel("Module5/Well_Data.xlsx"))

# View the top of the dataset
head(well_data) 

,Tax_ID,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1006004,9/24/12,52,165,41,60.0,7.7,ND
2,1024009,12/17/15,40,445,42,2.0,7.3,ND
3,1054019,2/2/15,45,160,40,40.0,7.4,ND
4,1057017,10/22/12,42,440,57,1.5,8.0,D
5,1060006,1/3/11,48,120,42,25.0,7.1,ND
6,1066006,12/15/15,60,280,32,10.0,8.2,D


The columns in this dataset are described below:
+ `Tax_ID`: Tax ID for the property
+ `Water_Sample_ID`: Date that the well was sampled 
+ `Casing_Depth`: Depth of the casing of the well (ft)
+ `Well_Depth`: Depth of the well (ft)
+ `Static_Water_Depth`: Static water depth in the well (ft)
+ `Flow_Rate`: Well flow rate (gallons per minute)
+ `pH`: pH of water sample
+ `Detect_Concentration`: Binary identifier (either non-detect (ND) or detect (D)) if As concentration detected in water sample 


## Training Module's Environmental Health Questions

This training module was specifically developed to answer the following environmental health questions:

1. Can we predict if iAs will be detected based on well water data? 

Before we dive into the code, let's get an understanding for what machine learning is, how to train a model, and assess its model performance.

## What is Machine Learning?

Machine learning is a field of study in computer science that involves creating algorithms, which are a set of instructions that perform a specific task on a given dataset. These algorithms enable researchers to create models that can automatically analyze to new and unforeseen situations capable of improving automatically through experience and data.

In other words, instead of being explicitly programmed to perform a task, a machine learning algorithm is designed to learn from examples and data, allowing it to adapt and improve over time. This approach is particularly useful for tasks that are too complex or difficult to be solved using traditional programming methods. **NOSA - I'M NOT SURE WHAT YOU MEANT HERE BY TRADITIONAL PROGRAMMING METHODS**

Through machine learning, scientists can:

+ Create a model that adapts to new circumstances 
+ Detect patterns in large and complex datasets
+ Evaluate the effectiveness of these patterns 
+ Make informed decisions about how to improve their models


## Types of Machine Learning

Within the field of machine learning, there are two types that are most commonly used in environmental toxicology research: supervised machine learning and unsupervised machine learning.

**Supervised machine learning** involves training a model using a labeled dataset, where each dependent or predictor variable is associated with an independent variable with a known outcome. This allows the model to learn how to predict the labeled outcome on data it hasn't "seen" before based on the patterns and relationships it previously identified in the data. For example, supervised machine learning has been used for cancer prediction and prognosis based on variables like tumor size, stage, and age ([Lynch et. al](https://www.sciencedirect.com/science/article/abs/pii/S1386505617302368?via%3Dihub), [Asadi et. al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7416093/)). 

Supervised machine learning includes: 
+ Classification: Using algorithms to classify a categorical outcome (ie. plant species, disease status, etc.)
+ Regression: Using algorithms to predict a continuous outcome (ie. temperature, chemical concentration, etc.)
<img src="https://github.com/UNC-CEMALB/P1006_Data-Organization-for-High-Dimensional-Analyses-in-Environmental-Health/assets/69641855/3c22a8fc-ada5-4199-9967-77f504d99d2a" width="684" />

([Soni, 2018](https://towardsdatascience.com/supervised-vs-unsupervised-learning-14f68e32ea8d))


**Unsupervised machine learning**, on the other hand, involves using models to find patterns or associations between variables in dataset that lack a known or labeled outcome. For example, unsupervised machine learning has been used to find associations of co-expressed genes within various biological pathways ([Botía et. al](https://bmcsystbiol.biomedcentral.com/articles/10.1186/s12918-017-0420-6), [Pagnuco et. al](https://www.sciencedirect.com/science/article/pii/S0888754317300575?via%3Dihub)).

<img src="https://github.com/UNC-CEMALB/P1006_Data-Organization-for-High-Dimensional-Analyses-in-Environmental-Health/assets/69641855/4f78adef-97b6-425f-9a51-43315b0fb7b2" width="684" />

([Langs et. al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6244522/))

It's worth noting that there are also other types of machine learning, including semi-supervised learning, reinforcement learning and deep learning, which won't be discussed in these training modules. Overall, the distinction between supervised and unsupervised learning is an important concept in machine learning, as it can inform the choice of algorithms and techniques used to analyze and make predictions from data.

To learn more check out the following resources: **I WILL LIKELY CHANGE THESE RESOURCES...I THINK WE CAN FIND BETTER ONES**

+ [Machine Learning Mastery](https://machinelearningmastery.com/machine-learning-in-r-step-by-step/)
+ [Master in Data Science](https://www.mastersindatascience.org/learning/machine-learning-algorithms/decision-tree/)
+ [IBM - What is Machine Learning](https://www.ibm.com/topics/machine-learning)
+ Machine Learning by Mueller, J. P. (2021). Machine learning for dummies. John Wiley &amp; Sons. 

## Training Supervised Machine Learning Models

In supervised machine learning, algorithms need to be trained before they can be used to predict data. This involves selecting a smaller portion of data, known as training data, so that the model will learn how to predict the outcome as accurately as possible. The process of training an algorithm is essential for enabling it to learn and improve over time, allowing it to make more accurate predictions and better adapt to new and changing circumstances. Ultimately, the effectiveness of a machine learning model depends on the quality and relevance of its training data.

Let's imagine you're interested in predicting an animal's species (either a cat or a dog) based on a dataset that contains variables regarding weight, height, coat color, ear shape, etc. These data can be divided into a training set and a test set. The **training set** is a subset of the data that the model will learn from to make associations between the predictor variables (ie. height, weight etc.) and the outcome (ie. cat or dog). The **test set** is used used to evaluate what the model has learned from the training set. 

Our dataset will look something like this with 60% of the data in the training set and 40% of the data in the test set:
<img src="https://github.com/UNC-CEMALB/P1006_Data-Organization-for-High-Dimensional-Analyses-in-Environmental-Health/assets/69641855/a8c723ec-e50c-4cb9-aff5-4230d0d6fc2c" width="684"/>
*Created with BioRender.com*


The process of developing a model involves dividing the data into three main sets:

1. **Training Set:** a subset of the data that the model uses to learn from the data by identifying patterns.

2. **Validation Set**: a subset of data that is used to evaluate the model's fit in an unbiased way by fine-tuning its parameters and optimizing its performance. This is akin to pop quizzes that help students improve their understanding and performance. **I DON'T THINK WE SHOULD WORRY ABOUT THIS, BECAUSE 9/10 WE WON'T HAVE ENOUGH DATA TO CREATE THIS IN ENVIRONMENTAL HEALTH RELATED RESEARCH AND WE HAVEN'T CREATED CODE THAT INCLUDES THIS**

2. **Test Set:** a subset of data that is used to evaluate the final model's fit based on the training and validation sets. This is the model's final exam, as it provides an objective assessment of the model's ability to generalize to new, unseen data.
  

It's important to note that the test set should only be used once, after the model has been trained using the training dataset. Using the test set multiple times during the development process can lead to overfitting, where the model performs well on the test data but poorly on new, unseen data. The ideal algorithm is generalizable or flexible enough to accurately predict unseen data. This is known as the bias-variance tradeoff. 

The last topic we should mention in this section is **cross validation** or **k-fold cross validation**. If the dataset is based on this 60:40 split that we mentioned earlier, our model's accuracy will likely be influenced based upon *where* this 60:40 split occurs in the dataset. Cross validation is implemented to ensure that the model is exposed to more patterns in the dataset, which reduces bias improving prediction accuracy. For example, if 5-fold cross validation were to be implemented, we would have 5 different folds or splits in the dataset with 4 retained for training and 1 retained for testing. These splits would occur 5 different times as shown in the figure below.

**INSERT IMAGE OF 5 FOLD CV**

Confusion matrix metrics would be calculated after each iteration and averaged for the final results. For additional information on the bias-variance tradeoff and the different types of cross validation check out the resources below. 

### Resources
+ [Understanding the Bias-Variance Tradeoff](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)
+ [Cross Validation in Machine Learning](https://towardsdatascience.com/cross-validation-in-machine-learning-72924a69872f)
+ [Cross Validation Pros & Cons](https://www.geeksforgeeks.org/cross-validation-machine-learning/)

## Assessing Model Performance  
Metrics from a confusion matrix are used to evalute model performance for classification-based supervised machine learning models. A confusion matrix consists of table that displays the numbers of how often the algorithm correctly or incorrectly predicted the outcome. 

Going back to our animal classification example, let's say the confusion matrix below is a result of how well the model was able to predict whether an animal was considered to be a cat or a dog.

<img src="https://github.com/UNC-CEMALB/P1006_Data-Organization-for-High-Dimensional-Analyses-in-Environmental-Health/assets/69641855/4c7a90a0-725b-48b8-ada4-77a5907131a0" width="684" />

*Created with BioRender.com*

Some of the metrics that can be obtained from a confusion matrix are listed below:

+ **Balanced Accuracy:** is the ratio of correct predictions (TP + TN) to the total number of predictions (TP + TN + TN + FN) and is typically used to assess overall model performance. This metric is prone to skew for imbalanced data. For example, if the animal dataset had 11 cats and 74 dogs the data would be considered to be imbalanced.

+ **Sensitivity or Recall:** evaluates how well the model was able to predict the "positive" class. It's the ratio of correctly classified true positives to total number of all true positives (TP + FN). 

+ **Specificity:** evaluates how well the model was able to predict the "negative" class. It's the ratio of correctly classified true negatives to total number of all true negatives (TN + FP). 

+  **Positive Predictive Value (PPV) or Precision:**  evaluates how well the model was able to predict the "positive" class. It's the ratio of correctly classified true positives to total number of predicted positives (TP + FP).

+  **Negative Predictive Value (PPV):**  evaluates how well the model was able to predict the "negative" class. It's the ratio of correctly classified true negatives to total number of predicted positives (TN + FN).

For all metrics, the values fall between 0 and 1, where 0 represents the model not being able to classify any data points correctly and 1 representing that the model was able to classify all test data correctly. Typically a balanced accuracy of at least 0.7 is considered respectable.


**Note**: For multiclass classification (more than two labeled outcomes to be predicted), the same metrics are used, but are obtained in a slightly different way. Regression based supervised machine learning models use loss functions to evaluate model performance. For more information regarding confusion matrices and loss functions for regression-based models check out the resources below.

### Resources
 + [Additional Confusion Matrix Metrics](https://medium.com/analytics-vidhya/what-is-a-confusion-matrix-d1c0f8feda5)
 + [Precision vs. Recall or Specificity vs. Sensitivity](https://towardsdatascience.com/should-i-look-at-precision-recall-or-specificity-sensitivity-3946158aace1)
 + [Loss Functions for Machine Learning Regression](https://towardsdatascience.com/understanding-the-3-most-common-loss-functions-for-machine-learning-regression-23e0ef3e14d3)

## Types of Supervised Machine Learning Algorithms

Although we'll be focusing on a random forest model in the coding example below, we want to mention other popular algorithims that can be used for supervised machine learning. These include: 

+ **K-Nearest Neighbors (KNN):** Uses Euclidean distance to classify a data point in the test set based upon the most common class of neighboring data points.
<img src="https://user-images.githubusercontent.com/96756991/232493057-1e7ce98b-6985-44cd-98a9-3cfea5994659.png" width="684" />
*Created with BioRender.com*

+ **Support Vector Machine (SVM):** Creates a decision boundary line (hyperplane) in n-dimensional space to seperate the data into each class so that when new data is presented they can be easily cateogrized. 
<img src="https://user-images.githubusercontent.com/96756991/233735220-08682ea4-fe13-41c8-8ac5-9ddda7859328.png" width="684" />

*Created with BioRender.com* 

+ **Random Forest:** Uses a multitude of decison trees trained on different parts of the training set and the classification of a data point in the test set is aggregated from all the trees. A **decision tree** 


### Resources
+ [K-Nearst Neighbor](https://www.ibm.com/topics/knn)
+ [Support Vector Machine](https://www.javatpoint.com/machine-learning-support-vector-machine-algorithm) **I LIKE THE TEXT IN THIS REFERENCE, BUT IT CONTAINS PYTHON CODE. NOT SURE IF THAT'S CONFUSING.**
+ [Random Forest](https://www.ibm.com/in-en/topics/random-forest)

Now that we have a better understanding of supervised machine learning, let's go ahead and write the code to predict iAs detection. 

### Changing Data Types 
`Water_Sample_Date` needs to be converted from a character to a date type, so that Random Forest understands this column contains dates. This will be done using the `lubridate` package.

In [15]:
well_data = well_data %>%
    # converting water sample date from a character to a date type 
    mutate(Water_Sample_Date = mdy(Water_Sample_Date))

head(well_data)

,Tax_ID,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration
,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1006004,2012-09-24,52,165,41,60.0,7.7,ND
2,1024009,2015-12-17,40,445,42,2.0,7.3,ND
3,1054019,2015-02-02,45,160,40,40.0,7.4,ND
4,1057017,2012-10-22,42,440,57,1.5,8.0,D
5,1060006,2011-01-03,48,120,42,25.0,7.1,ND
6,1066006,2015-12-15,60,280,32,10.0,8.2,D


### Testing for differences in predictor variables acrosss the outcome classes

It's useful to run summary statistics on the variables that will be used as predictors in the model to see if there are differences in distributions between the outcomes classes (either non-detect or detect in this case). Typically, greater signficance often leads to better predictivity, since the model is better able to separate the classes. We'll use the `tbl_summary()` function from the `gtsummary` package.

For more information on the `tbl_summary()` function, check out this helpful [Tutorial](https://www.danieldsjoberg.com/gtsummary/articles/tbl_summary.html).

In [16]:
well_data %>%
    tbl_summary(by = Detect_Concentration,
    # Selecting columns to include
    include = colnames(well_data[c(2:8)]), 
    # Displaying the mean and standard deviation in parantheses for all continuous variables
                statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
    # Adding a column that displays the total number of samples for each variable
    # This will be 713 for all variables since we have no missing data
    add_n() %>% 
    # Adding a column that dispalys the p value from anova
    add_p(test = list(all_continuous() ~ "aov")) %>% 
    as_tibble()

**Characteristic**,**N**,"**D**, N = 198","**ND**, N = 515",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-03-05 (957.843005291701),2013-06-05 (979.174260670888),0.3
Casing_Depth,713,55 (23),74 (33),<0.001
Well_Depth,713,334 (128),301 (144),0.005
Static_Water_Depth,713,36 (13),35 (12),0.4
Flow_Rate,713,14 (16),25 (33),<0.001
pH,713,7.82 (0.40),7.45 (0.55),<0.001


All the variables are very signficant with the exception of the sample date and the static water depth, therefore the model should predict fairly well.

## Setting up Cross Validation

As mentioned above, cross validation is implemented so that the model is trained and tested on different portions of the entire dataset. We'll use 5-fold cross validation, but note that *k* can be changed.

In [17]:
# Since the splits in the dataset are random, a seed is set for reproducibility to ensure the splits are occuring
# in the same locations each time the code is run
set.seed(12)

# 5-fold cross validation
# Saving the index (row number) where the 5 splits are occuring
# These indices will be iterated through using a loop to create each training and testing datasets
well_index = createFolds(well_data$Detect_Concentration, k = 5) 

In [ ]:
# Creating an empty dataframe to save the metics
metrics = data.frame()

# Iterating through the cross validation folds
for (i in 1:length(well_index)){
    # Training data
    data_train = well_data[-well_index[[i]],]
    
    # Test data
    data_test = well_data[well_index[[i]],]
    
    # Random forest
    reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])
}

## Working with the data 

**Download Packages**
```{r}
install.packages("tidyverse")
install.packages("pheatmap")
install.packages("ggplot2")
install.packages("reshape2")
install.packages("arsenal")
install.packages("superheat")

library(tidyverse)
library(pheatmap)
library(ggplot2)
library(reshape2)
library(arsenal)
library(superheat)
library(readxl)
library(gtsummary)
library(e1071)
library(Hmisc)
library(glmnet)
library(randomForest)
library(pROC)
library(lattice)
library(survival)
library(Formula)
library(caret)
```
**Set working directory**
```{r}
getwd()
setwd("/Users/ritaavenbuan/Desktop")
```

**Read in the Data & change sex into factors**

Changing Sex into factors will help the run the machine learning models run properly.

```{r}
#import the data and remove variables that are not going to be useful for your analysis
pre.dataset <- na.omit(read.csv("Proteomics_Imputed_PreExposureSubjects.csv")) %>%
  select(-SubjectID, -Race, -Ethnicity, -Age, -BMI) %>%
  drop_na() 

#change the class of Sex (from characters to factors)
class(pre.dataset$Sex)
pre.dataset$Sex <- as.factor(pre.dataset$Sex)
```

**Statistical Summary of the Data**

```{r}
pre.dataset %>%
  tbl_summary(by = "Sex", 
               statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>%
  add_p(test = list(all_continuous() ~ "aov")) %>%
  as_flex_table()


#post.dataset <-read.csv("Proteomics_Imputed_PostExposureSubjects.csv")
```

**Review the data**

```{r}
head(pre.dataset)
```

**Decision Tree**

A Decision Tree is a type of supervised machine learning model that makes predictions based on how a question was answered. 

![image](https://user-images.githubusercontent.com/96756991/228426001-9a73d4b5-017c-430b-b48f-0aa91dedc4ea.png)
Created with BioRender.com

- _Root node:_ Base of the Decision Tree 
- _Splitting:_ A node divided into sub-nodes 
- _Decision mode:_ A sub-node broken down into additional sub-nodes
- _Leaf node:_ A sub-node that does not split into additional sub-nodes (terminal node). It indicates a potential outcome. 
- _Pruning:_ Removing sub-nodes of a decision tree
- _Branch:_ A section of a decision tree with multiple nodes
- _Pruning:_ The process of removing sub-nodes of a decision tree

In this module, we will start by creating decision tree. 

[Reference](https://www.mastersindatascience.org/learning/machine-learning-algorithms/decision-tree/)

**Set data up for Reproducibility**
```{r}

#set up for reproducibility 
set.seed(15)
control_params <- rpart.control(minsplit = 5)

```

**Splitting data into testing and training set**

In this step we want to split the data into the training set (what the algorithm will use to learn data) and the test set. We also want to make sure that we are cross-validating here. 

_Cross-validate:_ The ability for the machine to predict new data and test its accuracy. We want to make sure our model is not overfitting (giving accurate results for predictions for training data but not new data). 
[Reference](https://learn.g2.com/cross-validation)

```{r}
sex_data_index = createFolds(pre.dataset$Sex, k = 5) #K in Cross Validation is usually 5 or 10 
errors = data.frame()
for (i in 1:length(sex_data_index)){
  sex_train = pre.dataset[-sex_data_index[[i]],]
  sex_test = pre.dataset[sex_data_index[[i]],]
  
  reg_tree = rpart(Sex ~., data = sex_train, method = "class", control = control_params)
  
  vim <- varImp(reg_tree)
  
  pred_tree <- predict(reg_tree, sex_test, type = "class", na.action = na.pass)
  
  cm_tree <- confusionMatrix(pred_tree, sex_test$Sex)
  
  accuracy_tree <- cm_tree$overall["Accuracy"]
}
```

**Preform Confusion Matrix**


```{r}
#perform confusion matrix 
# Set the number of folds for cross-validation
k <- 5

# Create the folds
folds <- cut(seq(1, nrow(pre.dataset)), breaks = k, labels = FALSE)

# Initialize an empty list to store the evaluation metrics for each fold
eval_metrics <- list()

# Loop over each fold
for (i in 1:k) {
  
  # Split the data into training and validation sets for this fold
  train_indices <- which(folds != i)
  valid_indices <- which(folds == i)
  sex_train <- pre.dataset[train_indices, ]
  sex_valid <- pre.dataset[valid_indices, ]
  
  # Fit the decision tree model on the training set
  reg_tree <- rpart(Sex ~., data = sex_train, method = "class", control = rpart.control(minsplit = 5))
  
  # Make predictions on the validation set
  pred_tree <- predict(reg_tree, sex_valid, type = "class")
  
  # Compute the evaluation metrics for this fold
  cm_tree <- confusionMatrix(pred_tree, sex_valid$Sex)
  eval_metrics[[i]] <- c(accuracy = cm_tree$overall['Accuracy'], 
                         sensitivity = cm_tree$byClass['Sensitivity'],
                         specificity = cm_tree$byClass['Specificity'])
}

# Calculate the average evaluation metrics across all folds
eval_metrics <- do.call(rbind, eval_metrics)
avg_eval_metrics <- colMeans(eval_metrics)

#confusion matrix results
print(cm_tree)
```
```{r}
Confusion Matrix and Statistics

          Reference
Prediction F M
         F 2 0
         M 3 1
                                          
               Accuracy : 0.5             
                 95% CI : (0.1181, 0.8819)
    No Information Rate : 0.8333          
    P-Value [Acc > NIR] : 0.9913          
                                          
                  Kappa : 0.1818          
                                          
 Mcnemar's Test P-Value : 0.2482          
                                          
            Sensitivity : 0.4000          
            Specificity : 1.0000          
         Pos Pred Value : 1.0000          
         Neg Pred Value : 0.2500          
             Prevalence : 0.8333          
         Detection Rate : 0.3333          
   Detection Prevalence : 0.3333          
      Balanced Accuracy : 0.7000          
                                          
       'Positive' Class : F               
```

Based on our confusion matrix, we have 50% accuracy which means that our model is accurately classifying proteins based on sex 50% of the time.  

**Visualize the Decision Tree**

We want to visualize the decision tree for the training data created above to see how the model is using the protemics data to categorize sex.

```{r}
reg_tree
rpart.plot(reg_tree, type = 2, extra = 101, under = TRUE, fallen.leaves = TRUE, main = "Decision Tree for Protein Differences by Sex")

cm_tree

table_matrix <- table(sex_test$Sex, pred_tree)
table_matrix
```
![image](https://user-images.githubusercontent.com/96756991/232521526-56740254-ad7e-4356-9d57-aa2c85914205.png)

Based on this decision tree, a classified protein in our proteomics training data set is CFP. When it is greater than 5540, the model predicts that the sex is female; however, when it is less than 5540, it uses another protein called BPIFB4. When BPIFB4 is greater than or equal to 6821, the model predicts the sex is female; else, it is male. 

Now, we are going to now run a more robust model Random Foreset to see if our accuracy increases and to see which proteins best predict sex. 

**Random Forest**

Random Forest creates many decision trees to reach an answer about the data set. 

![image](https://user-images.githubusercontent.com/96756991/228436169-c7fe932a-9aa4-41e2-b56c-9a8830b834a8.png)
Created with BioRender.com

[Reference](https://www.ibm.com/topics/random-forest) 

**Set data up for Reproducibility**
```{r}
rf_classification = function(pre.dataset, outcome, pred_outcome) {
  #setting for reproducibility
  set.seed(15)

```

**Split data into training and test sets**
```{r}
  #splitting data into training and testing sets 
  sex_data_index = createFolds(pre.dataset$Sex, k = 5) #K in Cross Validation is usually 5 or 10 
```

**Create an empty data frame for error, empty numeric vector for accuracy, sensitivity and specificity, and an empty list for importance**
```{r}

  errors = data.frame()
  accuracy = c()
  sensitivity = c()
  specificity = c()
  importance = list()
 ```
 
 **Run Random Forest and Cross-validation**
 ```{r}
  #set up cross valdiation loop using the sex_data_index created previously
  for (i in 1:length(sex_data_index)){
    sex_train = pre.dataset[-sex_data_index[[i]],]
    sex_test = pre.dataset[sex_data_index[[i]],]
    
    # Set up the grid of hyperparameters to search over
    ntree_values = c(50, 250, 500) #number of trees
    p = dim(pre.dataset)[2] - 1 #number of variables in dataset
    mtry_values = c(p/2, sqrt(p), p)
    
    # Set up the tuning grid for the random forest
    tuning_grid = expand.grid(ntree = ntree_values,
                              mtry = mtry_values)
    
    # Fit the random forest model with 5-fold cross-validation
    rf_model = train(as.factor(outcome) ~ ., 
                     data = sex_train, 
                     method = "rf", 
                     trControl = trainControl(method = "cv", 
                                              number = 5), 
                     tuneGrid = tuning_grid)
    
    # Make predictions on the test set
    rf_pred = predict(rf_model, newdata = sex_test)
    
    # Run confusion matrix for the test set
    cm = confusionMatrix(rf_pred, sex_test$Sex)
    accuracy[i] = cm$overall[1]
    sensitivity[i] = cm[2,2]/sum(cm[2,])
    specificity[i] = cm[1,1]/sum(cm[1,])
    
    # Save the variable importance for this fold
    importance[[i]] = varImp(rf_model)$importance
    
  }
  
  # Combine the evaluation metrics across all folds
  eval_metrics = data.frame(accuracy = mean(accuracy), 
                            sensitivity = mean(sensitivity), 
                            specificity = mean(specificity))
  
  # Run the mean importance of each variable across all folds
  variable_importance = data.frame(variable = names(pre.dataset)[-pred_outcome], 
                                   mean_importance = unlist(lapply(importance, 
                                                                   function(x) mean(x[,"MeanDecreaseGini"]))))
  
  # Return the evaluation metrics and variable importance
  return(list(eval_metrics = eval_metrics, variable_importance = variable_importance))
}
```

![image](https://user-images.githubusercontent.com/96756991/232552051-a6016e75-fd91-45fd-b237-056f1e1a56ea.png)

**Mean decrease accuracy**

Mean decrease accuracy tells us how important each feature is in making accurate predictions. A higher mean decrease accuracy score indicates that a feature is more important in making predictions, while a lower score means it's less important. With this particular dataset, we see that proteins such as DCXR, GBP6, HPR, PSME1, DNAJA4, GSTP1, DDT, KRT7, CTNNA1, and KRT18 are proteins that best predict sex differences.
[Reference](https://plos.figshare.com/articles/figure/Variable_importance_plot_mean_decrease_accuracy_and_mean_decrease_Gini_/12060105/1)

**Mean decrease Gini**

The mean decrease in Gini is based on the decrease in the Gini impurity index, and it measures how each predictor contributes to the purity of the nodes in the decision trees. In other words, the mean decrease Gini score tells us how important a feature is in making decisions in the dataset. The higher the score, the more important the feature is in making decisions and splitting the data. Our data shows that HPR, DCXR, PSMB7, KRT15, HYOU1, and GBP6 are essential proteins for making decisions in this dataset.
[Reference](https://www.analyticsvidhya.com/blog/2021/03/how-to-select-best-split-in-decision-trees-gini-impurity/)

**Random Forest Confusion Matrix**

```{r}
# Create folds for cross-validation
folds <- createFolds(pre.dataset$Sex, k = 5)

# Initialize empty vectors for storing results
accuracy <- c()
sensitivity <- c()
specificity <- c()

# Loop through each fold
for (i in 1:length(folds)) {
  # Split data into training and test sets
  train_data <- pre.dataset[-folds[[i]], ]
  test_data <- pre.dataset[folds[[i]], ]
  
  # Train random forest model on training data
  rf_model <- randomForest(Sex ~ ., data = train_data, importance = TRUE)
  
  # Make predictions on test data
  predictions <- predict(rf_model, test_data)
  
  # Calculate confusion matrix
  cm <- confusionMatrix(predictions, test_data$Sex)
  cm
  
  # Calculate and store metrics
  accuracy[i] <- cm$overall["Accuracy"]
  sensitivity[i] <- cm$byClass["Sensitivity"]
  specificity[i] <- cm$byClass["Specificity"]
}

# Print mean and standard deviation of metrics
cat("Mean Accuracy:", mean(accuracy), "±", sd(accuracy), "\n")
cat("Mean Sensitivity:", mean(sensitivity), "±", sd(sensitivity), "\n")
cat("Mean Specificity:", mean(specificity), "±", sd(specificity), "\n")

```

```{r}
Mean Accuracy: 0.4933333 ± 0.2349941 
Mean Sensitivity: 0.2666667 ± 0.2527625 
Mean Specificity: 0.65 ± 0.3555122 
```

By comparing the mean accuracy of the Random Forest Model to the Decision Tree, we find that the Decision Tree achieved higher accuracy with this dataset. This indicates that, for this specific dataset, a Decision Tree would be the more appropriate model to use for predicting our data. Considering the relatively low number of observations in the dataset (27), we may not have sufficient data to support the use of a robust Random Forest model.


## Test Your Knowledge 

Using the Post-exposure data provided, create a decision tree and random forest model to answer the following questions:

1. Can we predict sex based on protein expression?
2. Which proteins best predict sex? 
3. Which model is more appropriate for predicting the first two outcomes? 


## Conclusion 

In conclusion, this training module has provided an informative introduction to supervised machine learning using classification techniques in R. Machine learning is a powerful tool that can help researchers gain new insights and improve models to analyze complex datasets better. The example we've explored demonstrates the utility of machine learning models for better understanding data with numerous features.

This module also offers valuable resources that can serve as a guide to learning more about machine learning and its application to environmental health science research. By leveraging machine learning techniques, researchers can unlock new avenues for exploration and contribute to a deeper understanding of complex environmental health issues.